# DSCI 100 Section 009 Group 8 Project Proposal

## Introduction
### Background Information
The UEFA Champions League is one of the most presitigous and widely followed club football competitions. The competition brings elite football clubs from across Europe together in competition.The data we chose to analyze is about game statistics. We are using the keystats.csv dataset. Our data set contains serial number, player name, club name, position, and various game statistics such as distance traveled on the field, number of goals, number of dribbles, etc. Our goal is to use the game statistics like number of goals and distance traveled on the field to predict the positions of the players.

### Questions
Predict the position of the player that participated in UEFA Champions League 2021-2022 based on the goals and distance covered in the fields.

### Dataset Description
The dataset source is the UEFA player statistics for every player in the UEFA Champions League in the 2021-2022 season scraped by Azmine Toushik Wasi.

## Preliminary Exploratory Data Analysis
### Reading the Data
First of all, the data has been pushed to our group's github repository. Therefore, we can use the read_csv function to read all the data we need (our data is separated into several files). First of all, we need to initialize the libraries we are going to use

In [ ]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)
source('tests.R')
source('cleanup.R')

The dataset we will use are the key stats, defending, and distribution dataset. We will read them using the read_csv function

In [1]:
key_stats <- read_csv("https://raw.githubusercontent.com/JoelGunawan/dsci-100-009-group-8/main/data/key_stats.csv")
defending <- read_csv("https://raw.githubusercontent.com/JoelGunawan/dsci-100-009-group-8/main/data/defending.csv")
distribution <- read_csv("https://raw.githubusercontent.com/JoelGunawan/dsci-100-009-group-8/main/data/distributon.csv")

ERROR: Error in read_csv("https://raw.githubusercontent.com/JoelGunawan/dsci-100-009-group-8/main/data/key_stats.csv"): could not find function "read_csv"


### Cleaning and Wrangling
The data we have is separated into several different datasets, therefore we will merge the dataset together using the merge function, which will merge two different data frames based on a certain property

In [2]:
football <- merge(key_stats, defending, by = "player_name") |>
    merge(distribution, by = "player_name")
football

ERROR: Error in eval(expr, envir, enclos): object 'key_stats' not found


We will wrangle the data in several ways before we can use it. First of all, we will change the position to a factor because it is a categorical value. After that, the distance covered initially given is stored in a character value, but we need it as a numeric. Therefore, we will convert it to a numeric column. After that, we will remove all goalkeepers from our model, because the position of goalkeeper is concrete and predicting which players are goalkeepers are not of relevance to anyone because they are so distinct.

## Methods
### Variables and Columns Used

### Result Visualization Methods

## Expected outcomes and significance

###  Expected Findings 
The goal of the analysis is to predict the position of players who participated in the UEFA Champions League 2021-2022 based on their performance metrics, specifically goals scored and distance covered on the field. We anticipate finding patterns and relationships in the data that allow us to classify players into different positions using a K-Nearest Neighbors (KNN) classification model.

### Impact of Findings
Such findings can have several significant impacts. Firstly, they can provide insights into the key performance indicators for different player positions, aiding coaches and scouts in player recruitment and tactical strategies. Secondly, it can assist in the development of more accurate and data-driven player profiles for fans, analysts, and fantasy football enthusiasts. Lastly, it may contribute to enhancing the overall understanding of player roles and their relationships to performance metrics, potentially influencing player development and training methodologies.

### Future Questions
These findings may lead to several future questions and areas of research. For instance, it could prompt inquiries into how different factors, such as player age, playing style, or team tactics, influence the relationship between goals scored, distance covered, and player position. It could also lead to more sophisticated machine learning models that incorporate additional performance metrics for even more accurate position classification. Furthermore, this analysis could open up discussions on the evolving nature of player positions in modern football and how they are defined in the context of data-driven insights.